In [70]:
import json
import pandas as pd
import numpy as np
import sys
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
from collections import Counter
import Levenshtein
import re
import regex

tweets = pd.read_json("Data/gg2013.json") 
with open('names.txt',"r") as f:
    names = f.read()
print("Done")

Done


In [2]:
#### Global variables
awards = ["best screenplay - motion picture", "best director - motion picture",
          "best performance by an actress in a television series - comedy or musical",
          "best foreign language film", "best performance by an actor in a supporting role in a motion picture",
          "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
          "best motion picture - comedy or musical",
          "best performance by an actress in a motion picture - comedy or musical",
          "best mini-series or motion picture made for television",
          "best original score - motion picture", "best performance by an actress in a television series - drama",
          "best performance by an actress in a motion picture - drama", "cecil b. demille award",
          "best performance by an actor in a motion picture - comedy or musical", "best motion picture - drama",
          "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television",
          "best performance by an actress in a supporting role in a motion picture", "best television series - drama",
          "best performance by an actor in a mini-series or motion picture made for television",
          "best performance by an actress in a mini-series or motion picture made for television",
          "best animated feature film", "best original song - motion picture",
          "best performance by an actor in a motion picture - drama", "best television series - comedy or musical",
          "best performance by an actor in a television series - drama",
          "best performance by an actor in a television series - comedy or musical"] 

key_word_award_mapping = {
        "best screenplay - motion picture": ["screenplay", "motion", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best director - motion picture": ["director", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actress in a television series - comedy or musical" : ["actress", "comedy|musical", "television|tv|tele|series", "^(?!.*support).*$"],
        "best foreign language film" : ["foreign"],
        "best performance by an actor in a supporting role in a motion picture" : ["actor", "support", "motion", "^(?!.*series).*$", "^(?!.*tv).*$"],
        "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television" : ["actress", "support", "television|tv|tele|series|film", "mini|film"],
        "best motion picture - comedy or musical" : ["motion", "comedy|musical", "^(?!.*actor).*$", "^(?!.*actress).*$", "^(?!.*series).*$", "^(?!.*tv).*$"],
        "best performance by an actress in a motion picture - comedy or musical" : ["actress", "comedy|musical", "^(?!.*support).*$", "^(?!.*series).*$", "^(?!.*tv).*$"],
        "best mini-series or motion picture made for television" : ["television|tv|series", "mini|film", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best original score - motion picture" : ["score", "^(?!.*actor).*$", "^(?!.*actress).*$", "^(?!.*by).*$"],
        "best performance by an actress in a television series - drama" : ["actress", "television|tv|tele|series", "drama", "^(?!.*support).*$"],
        "best performance by an actress in a motion picture - drama" : ["actress", "drama", "^(?!.*support).*$", "^(?!.*series).*$"],
        "cecil b. demille award" : ["cecil", "demille"],
        "best performance by an actor in a motion picture - comedy or musical" : ["actor", "comedy|musical", "^(?!.*support).*$", "^(?!.*series).*$", "^(?!.*tv).*$"], 
        "best motion picture - drama" : ["motion", "drama", "^(?!.*actor).*$", "^(?!.*actress).*$", "^(?!.*series).*$", "^(?!.*tv).*$"],
        "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television" : ["support", "actor", "television|tv|tele|series|film", "mini|film"],
        "best performance by an actress in a supporting role in a motion picture" : ["actress", "support", "motion", "^(?!.*series).*$", "^(?!.*tv).*$"],
        "best television series - drama" : ["television|tv|series", "drama", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actor in a mini-series or motion picture made for television": ["actor", "television|tv|series", "mini|film" "^(?!.*support).*$"],
        "best performance by an actress in a mini-series or motion picture made for television" : ["actress", "television|tv|tele|series|film", "mini|film", "^(?!.*support).*$"],
        "best animated feature film" : ["animated", "film|feature"],
        "best original song - motion picture" : ["song", "^(?!.*actor).*$", "^(?!.*actress).*$", "^(?!.*by).*$"],
        "best performance by an actor in a motion picture - drama" : ["actor", "drama", "^(?!.*support).*$", "^(?!.*series).*$", "^(?!.*tv).*$"],
        "best television series - comedy or musical" : ["television|tv|series", "comedy|musical", "^(?!.*actor).*$", "^(?!.*actress).*$"],
        "best performance by an actor in a television series - drama" : ["actor", "television|tv|series", "drama", "^(?!.*support).*$"],
        "best performance by an actor in a television series - comedy or musical" : ["actor", "television|tv|series", "comedy|musical", "^(?!.*support).*$"]    } 
    
tokens_to_ignore = ["RT", "@", "golden globe", "goldenglobe", "globe", "globes", "golden", "hosts", "http", "#"]

In [3]:
###Sequence to Get Potential Answers
def getSolutionsFromPositions(doc, pos):
    consecutives = []
    current = []
    for elt in pos:
        if len(current) == 0:
            current.append(elt)
        else:
            if current[-1] == elt - 1:
                current.append(elt)
            else:
                consecutives.append(current)
                current = [elt]
        if (len(current) != 0):
            consecutives.append(current)
    if ([doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives] is None):
        return []
    else:
        return [doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives]
    
def checkIfPerson(h):
    isPerson = False
    for ent in h.ents:
        if ent.label_ == "PERSON" or h.text.lower().split()[0] + "\n" in names.lower(): #Includes Will because it isn't recognized as a name
            isPerson = True
    return isPerson

In [4]:
def host():
    print("starting")
    hosts = {}
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        if ("host" in tweet.lower()):
            doc = nlp(tweet)
            #Grabs proper pronouns (Removes certain key phrases too)
            pos = [tok.i for tok in doc if (tok.pos_ == "PROPN" and not any(i in tok.text.lower() for i in tokens_to_ignore))]
            properPronouns = getSolutionsFromPositions(doc, pos)
            
            # Only want Phrases with 2 words (to represent a full name)
            for h in (properPronouns):
                if not checkIfPerson(h):
                    continue
                lowerCase = h.text.lower()
                if (len(h.text.split()) != 2):
                    continue
                if lowerCase in hosts:
                    hosts[lowerCase] += 1
                else:
                    hosts[lowerCase] = 1
                        
    #Find the top 1-2 Names, those will be the people mentioned the most with the word host
    host_list = sorted(hosts.keys(), key=hosts.get, reverse=True)
    p = np.percentile(list(hosts.values()),98)
    answer = []
    print(host_list)
    for key in host_list:
        if (hosts[key] > p):
            answer.append(key)
    if (len(answer) >= 2):
        return answer[:2]
    else:
        return answer[0]
    

In [338]:
print(host())

starting
{'amy poehler': 708, 'jay leno': 11, 'tina fey': 577, 'amy pohler': 42, 'ryan seacrest': 4, 'princess jasmine': 2, 'amy poeler': 6, 'bill murray': 9, 'tina faye': 3, 'knew amy': 3, 'james cameron': 14, 'james franco': 10, 'tina fay': 5, 'quentin tarantino': 33, 'al roker': 2, 'amy polard': 2, 'ron riggle': 2, 'amy p.': 3, 'anne hathaway': 16, 'amy poehlr': 2, 'host guiliana': 2, 'matt pinfield': 2, 'amy phoehler': 2, 'baby momma': 2, 'seth macfarlane': 9, 'jessica lange': 2, 'selma hayek': 2, 'paul rudd': 15, 'tony mendez': 5, 'zooey deschanel': 2, 'daniel craig': 3, 'jennifer lawrence': 11, 'bill clinton': 13, 'el clinton': 2, 'mr. clinton': 2, 'maggie smith': 18, 'rebel wilson': 3, 'fat amy': 2, 'maggie smi': 14, 'will ferrel': 3, 'kristin wiig': 19, 'taylor swift': 7, 'ricky gervais': 24, 'kristen wiig': 199, 'will farrell': 15, 'will ferrell': 7, 'ellen degeneres': 2, 'kristen wig': 10, 'kristin wigg': 2, 'kristen wigg': 9, 'quentin tarantin': 30, 'quentin tara': 2, 'amy h

In [14]:
### Not Working Well
def award_names():
    print("STARTING")
    awardNames = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        if ("best " in tweet.lower()):# and(("nominated" in tweet.lower()) or ("won" in tweet.lower()))):                       
            doc = nlp(tweet)
            nouns = ["PROPN", "NOUN"]
            pos = [tok.i for tok in doc if ((tok.pos_ in nouns) or (tok.pos_ == "ADJ" and tok.text.lower() == "best")) and not any(i in tok.text.lower() for i in tokens_to_ignore)]
            words = getSolutionsFromPositions(doc, pos)
            for h in (words):
                lowerCase = h.text.lower()
                if (lowerCase[0:5] != "best "):
                    continue
                if lowerCase in awardNames:
                    awardNames[lowerCase] += 1
                else:
                    awardNames[lowerCase] = 1
                        
                                            
    #Find the top 1-2 Names, those will be the people mentioned the most with the word best
    award_list = sorted(awardNames.keys(), key=awardNames.get, reverse = True)
    answer = []
    for i in award_list:
        answer.append(i)
    return answer[:27]

In [209]:
print(award_names())

STARTING
['best actor', 'best actress', 'best director', 'best motion picture', 'best picture', 'best supporting actress', 'best supporting actor', 'best original song', 'best screenplay', 'best tv series', 'best song', 'best part', 'best motion picture drama', 'best performance', 'best drama', 'best movie', 'best comedy', 'best speech', 'best exotic marigold hotel', 'best television series', 'best foreign film', 'best show', 'best thing', 'best actress drama winner', 'best moments', 'best acceptance speech', 'best foreign language film']


In [ ]:
def fuzzy_match(base, candidate, threshold):
    dist = Levenshtein.distance(base, candidate)
    base_len = len(base_str)
    return (dist <= round(base_len * threshold))

In [27]:
def award_winners():
    print("STARTING")
    phrases_to_ignore = ["motion picture", "best", "hbo"]
    winners = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
         
        if (("won" in tweet.lower() or "congrat" in tweet.lower() or "winner" in tweet.lower() or "goes to" in tweet.lower()) and ("best" in tweet.lower() or "cecil" in tweet.lower())):
            doc = nlp(tweet)
            for award in awards:
                go = True
                for i in key_word_award_mapping[award]:
                    if not bool(re.search(i, tweet.lower())):
                        go = False
                        break
                if not go:
                    continue
                
                movies_tv_pos = ["PROPN", "NOUN", "ADJ", "AUX", "VERB"]
                if ("act" in award or "cecil" in award):
                    pos = [tok.i for tok in doc if (tok.pos_ == "PROPN" and not any(i in tok.text.lower() for i in tokens_to_ignore))] #maybe include not congrat
                else:
                    pos = [tok.i for tok in doc if ((tok.pos_ in movies_tv_pos or tok.text.lower() == "the" or tok.text.lower() == "of") and not any(i in tok.text.lower() for i in tokens_to_ignore))]
            
    
            
            
                properPronouns = getSolutionsFromPositions(doc, pos)
                for h in (properPronouns):
                    if ("hbo" in h.text.lower() or "best" in h.text.lower() or "motion picture" in h.text.lower() or "winners" == h.text.lower() or "tv" == h.text.lower()):
                        continue
                    if (h.text.lower() in award):
                        continue
                    if (("director" in award or "cecil" in award or "act" in award) and (len(h.text.split()) != 2 and len(h.text.split()) != 3) ):
                        continue
                    #if (("director" in award or "cecil" in award or "act" in award) and not checkIfPerson(h)):
                    #    continue
                    if award in winners:
                        if h.text.lower() in winners[award]:
                            winners[award][h.text.lower()] += 1
                        else:
                            winners[award][h.text.lower()] = 1
                    else:
                        winners[award] = {}
                        winners[award][h.text.lower()] = 1

    winn = {}
    for x in winners:
        winn[x] = max(winners[x], key=winners[x].get)
    print("done")
#     print(winners['best performance by an actor in a motion picture - drama'])
#     print(winners['best performance by an actor in a television series - drama'])
    return winn
#     for i in awards:
#         if i not in winn.keys():
#             print(i)
award_winners()
# q = award_winners()

STARTING
done


{'best performance by an actor in a motion picture - drama': 'eddie redmayne',
 'best motion picture - drama': 'boyhood',
 'best director - motion picture': 'richard linklater',
 'cecil b. demille award': 'george clooney',
 'best performance by an actress in a television series - drama': 'ruth wilson',
 'best performance by an actress in a motion picture - drama': 'julianne moore',
 'best original song - motion picture': 'glory',
 'best television series - comedy or musical': 'transparent',
 'best mini-series or motion picture made for television': 'fargo',
 'best animated feature film': 'train',
 'best foreign language film': 'leviathan',
 'best performance by an actor in a supporting role in a motion picture': 'j.k. simmons',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television': 'matt bomer',
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': 'joanne frog

In [6]:
def presenters():
    print("STARTING")
    phrases_to_ignore = ["motion picture", "best", "hbo"]
    presenters = {}
    count = 0
    for x, col in tweets.iterrows():
        tweet = col["text"]
        if "RT" in tweet: #Prevents getting the same tweet multiple times
            continue
        
#         if "aziz ansari" in tweet.lower():
#             print(tweet)
        test = ["best performance by an actress in a television series - comedy or musical",
"best mini-series or motion picture made for television",
"best performance by an actor in a motion picture - comedy or musical",
"best performance by an actor in a supporting role in a series, mini-series or motion picture made for television",
"best performance by an actress in a supporting role in a motion picture",
"best performance by an actor in a mini-series or motion picture made for television",
"best performance by an actress in a mini-series or motion picture made for television",
"best performance by an actor in a motion picture - drama",
"best performance by an actor in a television series - drama",
"best performance by an actor in a television series - comedy or musical"]
        
        
        if (("present" in tweet.lower() or "announc" in tweet.lower() or "give" in tweet.lower() or "award" in tweet.lower()) and ("best" in tweet.lower() or "cecil" in tweet.lower())):
            doc = nlp(tweet)
            for award in awards:
                go = True
                for i in key_word_award_mapping[award]:
                    if not bool(re.search(i, tweet.lower())):
                        go = False
                        break
                if not go:
                    continue
                pos = [tok.i for tok in doc if ((tok.pos_ == "PROPN" or tok.text.lower() == "will") and not any(i in tok.text.lower() for i in tokens_to_ignore))] #maybe include not congrat
                properPronouns = getSolutionsFromPositions(doc, pos)
#                 print(tweet)
#                 print(properPronouns)
#                 print(award)
#                 print()
                for h in (properPronouns):
                    if not checkIfPerson(h):
                        continue
                    if ("hbo" in h.text.lower() or "best" in h.text.lower() or "motion picture" in h.text.lower() or "winners" == h.text.lower() or "award" in h.text.lower() or "tv" in h.text.lower()):
                        continue
                    if (h.text.lower() in award):
                        continue
                    if (len(h.text.split()) != 2 and len(h.text.split()) != 3):
                        continue
                    if award in presenters:
                        if h.text.lower() in presenters[award]:
                            presenters[award][h.text.lower()] += 1
                        else:
                            presenters[award][h.text.lower()] = 1
                    else:
                        presenters[award] = {}
                        presenters[award][h.text.lower()] = 1

    winn = {}
    for y in awards:
        if y == 'best mini-series or motion picture made for television':
            continue
        if q[y] in presenters[y]:
            presenters[y][q[y]] = 0
    
    for x in presenters:
        winn[x] = sorted(presenters[x], key=presenters[x].get, reverse = True)[:2]
    print("done")
    #print(presenters['best performance by an actress in a motion picture - comedy or musical'])
    print(winn)
    for i in awards:
        if i not in winn.keys():
            print(i)
            
            
presenters()

STARTING


NameError: name 'q' is not defined

In [219]:
from nltk.corpus import stopwords
def get_human_names(text):
    
    people = []
    human_names = []
    stopwordsList = stopwords.words('english') + ['GoldenGlobes', 'Golden', 'Globes', 'Golden Globes', 'RT', 'VanityFair', 'golden', 'globes' '@', 'I', 'we', 'http', '://', '/', 'com', 'Best', 'best', 'Looking','Nice', 'Most', 'Pop', 'Hip Hop', 'Rap', 'We', 'Love', 'Awkward','Piece', 'While', 'Boo', 'Yay', 'Congrats', 'And', 'The', 'Gq', 'Refinery29', 'USWeekly', 'TMZ', 'Hollywood', 'Watching', 'Hooray', 'That', 'Yeah', 'Can', 'So', 'And', 'But', 'What', 'NShowBiz', 'She', 'Mejor', 'Did', 'Vanity', 'Fair', 'Drama', 'MotionPicture', 'News', 'Take', 'Before', 'Director', 'Award', 'Movie Award', 'Music Award', 'Best Director', 'Best Actor', 'Best Actress', 'Am', 'Golden Globe', 'Globe', 'Awards', 'It']
    
    for i in range(len(text)):
        people=[ee for ee in text[i].ents if ee.label_ == 'PERSON']
    
    for curr in people:
        if curr.text not in stopwordsList:
            human_names.append(curr)

    return human_names

def get_nominees():
    '''Nominees is a dictionary with the hard coded award
    names as keys, and each entry a list of strings. Do NOT change
    the name of this function or what it returns.'''
    print('STARTIING')

    nominees_tweets = []
    nominees = dict()
    nominee_words = ['nominee', 'nominees', 'nominating', 'nominated', 'nominates', 'Nominee', 'Nominees', 'Nominating', 'Nominated', 'Nominates']
    
    #generic words that are likely to appear that will not be human names
    ignore = ['Fair', 'Best', 'She', 'He', 'Hooray' 'Supporting', 'Actor', 'Actress', 'The', 'A', 'Life', 'Good', 'Not', 'Drinking', 'Eating', 'And', 'Hooray', 'Nshowbiz', 'TMZ', 'VanityFair', 'People', 'CNN', 'CBS', 'Magazine', 'Television', 'Mejor', 'Better', 'Score', 'Movie', 'Film', 'Picture', 'All', 'This', 'That', 'Anyway', 'However', 'Song', 'Tune', 'Music', 'Drama', 'Comedy', 'So', 'Better', 'Netflix', 'Someone', 'Mc', 'Newz', 'Season', 'Should', 'Fashion', 'Has', 'How', 'Oscar', 'Grammy', 'Oscars', 'Oscars', 'Drink', 'Because', 'Interesting', 'Although', 'Though', 'Yay', 'Congrats']

    # temporary winners list 
    winners = {'cecil b. demille award' : 'Jodie Foster', 'best motion picture - drama' : 'Argo', 'best performance by an actress in a motion picture - drama' : 'Jessica Chastain', 'best performance by an actor in a motion picture - drama' : 'Daniel Day-Lewis', 'best motion picture - comedy or musical' : 'Les Miserables', 'best performance by an actress in a motion picture - comedy or musical' : 'Jennifer Lawrence', 'best performance by an actor in a motion picture - comedy or musical' : 'Hugh Jackman', 'best animated feature film' : 'Brave', 'best foreign language film' : 'Amour', 'best performance by an actress in a supporting role in a motion picture' : 'Anne Hathaway', 'best performance by an actor in a supporting role in a motion picture' : 'Christoph Waltz', 'best director - motion picture' : 'Ben Affleck', 'best screenplay - motion picture' : 'Quentin Tarantino', 'best original score - motion picture' : 'Mychael Danna', 'best original song - motion picture' : 'Skyfall', 'best television series - drama' : 'Homeland', 'best performance by an actress in a television series - drama' : 'Claire Danes', 'best performance by an actor in a television series - drama' : 'Damian Lewis', 'best television series - comedy or musical' : 'Girls', 'best performance by an actress in a television series - comedy or musical':'Lena Dunham', 'best performance by an actor in a television series - comedy or musical':'Don Cheadle', 'best mini-series or motion picture made for television':'Game Change', 'best performance by an actress in a mini-series or motion picture made for television':'Julianne Moore', 'best performance by an actor in a mini-series or motion picture made for television':'Kevin Costner', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': 'Maggie Smith', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television': 'Ed Harris'}
    # winners = award_winners()
    
    for award in awards:
        nominees[award] = []

    types = re.compile('.*(hop(ed|ing|e|es))\s(@)?(\w+)\s(w(o|i)(n|ns|nning)).*', re.IGNORECASE)

    # retrieve tweets for nominees
    for curr in range(0, 3000):
        doc = nlp(tweets['text'][curr])
        if any(word in doc.text for word in nominee_words):
            nominees_tweets.append(doc)
        elif re.search(types, ' '.join(doc.text)):
            nominees_tweets.append(doc)
            
    for curr in nominees:
        award_tweets = []
        award_counter = Counter()

        temp = winners[curr].split(' ')

        for tweet in nominees_tweets:
            for winner in temp:
                if winner in tweet.text:
                    award_tweets.append(tweet)

        tweet_names = get_human_names(award_tweets) #get human names per tweet
        
        for i in range(len(tweet_names)):
            for name in tweet_names[i].text: #count each name
                if name in winners[curr] or name in ignore or name in nominee_words:
                    pass
                else:
                    award_counter[name] += 1

        final_nominees = []
        for key,v in award_counter.most_common(5):
            key_final = key.encode("utf-8")
            final_nominees.append(key_final)

        nominees[curr] = final_nominees

    return nominees


In [220]:
get_nominees()

STARTIING


{'best screenplay - motion picture': [],
 'best director - motion picture': [b'N', b'o', b'm', b'i'],
 'best performance by an actress in a television series - comedy or musical': [],
 'best foreign language film': [],
 'best performance by an actor in a supporting role in a motion picture': [],
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': [],
 'best motion picture - comedy or musical': [],
 'best performance by an actress in a motion picture - comedy or musical': [],
 'best mini-series or motion picture made for television': [b'r',
  b'J',
  b'i',
  b'f',
  b'L'],
 'best original score - motion picture': [],
 'best performance by an actress in a television series - drama': [],
 'best performance by an actress in a motion picture - drama': [b'r',
  b'f',
  b'L',
  b'w'],
 'cecil b. demille award': [],
 'best performance by an actor in a motion picture - comedy or musical': [],
 'best motion picture - drama': [],
